# Default Import

In [13]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2020-06-30 16:45 | by soge | Using ris library version: 1.5.3 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\ConED FDR

# Db connections

In [2]:
try: 
    db.params['user']
except:
    sqldb = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

	Missing SQL Server Native Client 10.0                     datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-06-30 13:53:07
- Query run 2020-06-30 13:53:07.291000
 Query time: Query run in 1000 microseconds 
 * Returned 0 rows *
	Missing SQL Server Native Client 10.0                     datetime2 will not be interpreted correctly



In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

In [4]:
sqldb.connect()

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-06-30 13:53:07


In [5]:
inj = sqldb.dfquery(""" WITH data AS (
                        SELECT DISTINCT ROW_WID, ACCIDENT_DT, OFF_STREET
                        FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
                        WHERE OFF_STREET in ('907LX4M11068','907LX4M11069')
                        and YEAR(core.ACCIDENT_DT) >2016
                        --and ((ACCIDENT_DESC  like '%con%') or (ACCIDENT_DESC  like '%driveway%') or (ACCIDENT_DESC  like '%plant%'))
                        )

                        SELECT YEAR(ACCIDENT_DT) AS yr
                               ,count(CASE WHEN OFF_STREET = '907LX4M11068' THEN (ROW_WID) END) '907LX4M11068'
                               ,count(CASE WHEN OFF_STREET = '907LX4M11069' THEN (ROW_WID) END) '907LX4M11069'
                        FROM data
                        GROUP BY YEAR(ACCIDENT_DT)
                        """)

In [6]:
inj

,yr,907LX4M11068,907LX4M11069
0,2017,21,12
1,2018,9,6
2,2019,12,11
3,2020,2,2


In [7]:
 data = sqldb.dfquery("""
                        WITH data AS (
                        SELECT *
                        FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
                        WHERE OFF_STREET in ('907LX4M11068','907LX4M11069')
                        and YEAR(core.ACCIDENT_DT) >2016
                        and ((ACCIDENT_DESC  like '%con%') or (ACCIDENT_DESC  like '%driveway%') or (ACCIDENT_DESC  like '%plant%') or (ACCIDENT_DESC  like '%barrier%') or (ACCIDENT_DESC  like '%doors%'))
                        )


                        ,data2 AS(
                        SELECT core.*, veh.PROPERTY_DAMAGED_DESC
                        FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
                        JOIN FORMS.dbo.WC_ACCIDENT_VEHICLE_F veh 
                        ON core.integration_id = veh.accident_id
                        WHERE core.OFF_STREET in ('907LX4M11068','907LX4M11069')
                        and YEAR(core.ACCIDENT_DT) >2016
                        and veh.[PROPERTY_DAMAGED_DESC] is not null
                        )


                        SELECT INTEGRATION_ID, ACCIDENT_DESC as DESCRIPTION
                        FROM data


                        UNION

                        SELECT INTEGRATION_ID, PROPERTY_DAMAGED_DESC as DESCRIPTION
                        FROM data2
                         """)

In [8]:
data

,INTEGRATION_ID,DESCRIPTION
0,0018917019,At tpo driver vehicle 2 stated that his vehicl...
1,0023618019,DAMAGE TO MEDIAN DIVIDER.
2,0023618019,Driver#1 states she was sideswiped by an unkno...
3,0034520019,"At T/P/O, unknown operator did enter the pedes..."
4,0034520019,GUARD RAIL AND CONCRETE BARRIER ON PEDESTRIAN ...
5,0043319019,OPERATOR VEHICLE ONE NB FDR CENTER LANE OF THR...
6,0107219019,AT TPO V1 STATES HE WAS TRAVELING N/B IN THE L...
7,0152919019,AT TPO DRIVER OF VEHICLE 1 STATES HE WAS TRAVE...
8,0154818019,"UPON ARRIVAL OF SCENE, VEHICLES HAD EXITED HIG..."
9,0173919019,AT T/P/O MV#1 STATED SHE WAS TRAVELING NORTHBO...


In [10]:
writer = pd.ExcelWriter(r'Outputs\coned_fdr_{}.xlsx'.format(ts), engine='xlsxwriter')    
inj.to_excel(writer, sheet_name='coned_fdr_injury_summary', index= False)
data.to_excel(writer, sheet_name='coned_fdr_accident_desc', index= False)
writer.save() 

In [14]:
ris.__version__

'1.5.3'